In [1]:
import json
import pprint

In [2]:
p = pprint.PrettyPrinter(indent=1)

In [3]:
f = open("assessment-attempts-20180128-121051-nested.json","r")

In [4]:
s = f.read()

In [5]:
json_data = json.loads(s)

In [6]:
f.close()

In [7]:
len(json_data)

3280

In [8]:
p.pprint(json_data[0])

{'base_exam_id': '37f0a30a-7464-11e6-aa92-a8667f27e5dc',
 'certification': 'false',
 'exam_name': 'Normal Forms and All That Jazz Master Class',
 'keen_created_at': '1516717442.735266',
 'keen_id': '5a6745820eb8ab00016be1f1',
 'keen_timestamp': '1516717442.735266',
 'max_attempts': '1.0',
 'sequences': {'attempt': 1,
               'counts': {'all_correct': False,
                          'correct': 2,
                          'incomplete': 1,
                          'incorrect': 1,
                          'submitted': 4,
                          'total': 4,
                          'unanswered': 0},
               'id': '5b28a462-7a3b-42e0-b508-09f3906d1703',
               'questions': [{'id': '7a2ed6d3-f492-49b3-b8aa-d080a8aad986',
                              'options': [{'at': '2018-01-23T14:23:24.670Z',
                                           'checked': True,
                                           'correct': True,
                                           'id': '

In [9]:
def recursive_walk_json_object(j, level):
    """recursively walk through a json object to explore the structure
       dictionaries will be put in alphabetic order to match the pretty print above"""
    
    level += 1
    
    if type(j) is dict:
        dict_2_list = list(j.keys())
        dict_2_list.sort()
        for k in dict_2_list:
            print("   " * level + "L" + str(level), k)
            recursive_walk_json_object(j[k], level)
    
    elif type(j) is list:
        for (i, l) in enumerate(j):
            print("  " * level + "  [" + str(i) + "]")
            recursive_walk_json_object(l, level)
            
    else:
        print("   " * level + " value:", j)

In [10]:
recursive_walk_json_object(json_data[0], -1)

L0 base_exam_id
    value: 37f0a30a-7464-11e6-aa92-a8667f27e5dc
L0 certification
    value: false
L0 exam_name
    value: Normal Forms and All That Jazz Master Class
L0 keen_created_at
    value: 1516717442.735266
L0 keen_id
    value: 5a6745820eb8ab00016be1f1
L0 keen_timestamp
    value: 1516717442.735266
L0 max_attempts
    value: 1.0
L0 sequences
   L1 attempt
       value: 1
   L1 counts
      L2 all_correct
          value: False
      L2 correct
          value: 2
      L2 incomplete
          value: 1
      L2 incorrect
          value: 1
      L2 submitted
          value: 4
      L2 total
          value: 4
      L2 unanswered
          value: 0
   L1 id
       value: 5b28a462-7a3b-42e0-b508-09f3906d1703
   L1 questions
      [0]
         L3 id
             value: 7a2ed6d3-f492-49b3-b8aa-d080a8aad986
         L3 options
          [0]
               L5 at
                   value: 2018-01-23T14:23:24.670Z
               L5 checked
                   value: True
               L

In [11]:
!cat assessment-attempts-20180128-121051-nested.json | jq ' .[] | .exam_name' -c | sort | uniq -c | sort -r

    394 "Learning Git"
    162 "Introduction to Python"
    158 "Introduction to Java 8"
    158 "Intermediate Python Programming"
    128 "Learning to Program with R"
    119 "Introduction to Machine Learning"
    109 "Software Architecture Fundamentals Understanding the Basics"
     95 "Beginning C# Programming"
     85 "Learning Eclipse"
     80 "Learning Apache Maven"
     79 "Beginning Programming with JavaScript"
     77 "Mastering Git"
     75 "Introduction to Big Data"
     67 "Advanced Machine Learning"
     59 "Learning Linux System Administration"
     58 "JavaScript: The Good Parts Master Class with Douglas Crockford"
     57 "Learning SQL"
     53 "Practical Java Programming"
     52 "HTML5 The Basics"
     51 "Python Epiphanies"
     48 "Software Architecture Fundamentals Beyond The Basics"
     43 "Introduction to Data Science with R"
     43 "Intermediate C# Programming"
     40 "Learning DNS"
     35 "Learning C# Best Practices"
     35 "Expert Data Wrangling with R"
 